In [3]:
import os
import glob
import pandas as pd
import re
import dash
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

df2 = pd.read_csv("clinician.csv")

df2 = df2.sort_values(['meta__person__unique_id', 'timestamp'])

def hips(playerid):
    one_athlete = df2[df2['meta__person__unique_id'].str.contains(playerid)]
    df_hips = one_athlete.filter(regex = re.compile(r'meta__person__unique_id|meta__session__session_guid|timestamp|hip|HIP|Hip'))
    
    lines = []

    for column in df_hips.columns[5:20]:
        trace = go.Scatter(
            x=df_hips['timestamp'],
            y=df_hips[column],
            mode='lines',
            name=column
        )
        lines.append(trace)

    layout = go.Layout(
        title='Clinician Hips',
        xaxis=dict(title='Timestamp'),
        yaxis=dict(title='Value')
    )

# Create the figure and plot it
    fig = go.Figure(data=lines, layout=layout)
    fig.show()
    return fig


# Define the app layout
app = dash.Dash(__name__, external_stylesheets=['https://codepen.io/chriddyp/pen/bWLwgP.css'])
app.layout = html.Div([
    html.H1("Tales of Dunk and Leg"),
    dcc.Dropdown(
        id='playerid-dropdown',
        options=[{'label': i, 'value': i} for i in df2['meta__person__unique_id'].unique()],
        value=df2['meta__person__unique_id'].unique()[0]
    ),       
    html.H2("Hips"),
    dcc.Graph(id='hips-graph'),
    html.H2("Knees"),
    #dcc.Graph(id='my-graph',figure=scatter),
    html.H2("Ankles"),
    #dcc.Graph(id='my-graph',figure=scatter)      
])


@app.callback(
    dash.dependencies.Output('hip-graph', 'figure'),
    [dash.dependencies.Input('playerid-dropdown', 'value')])

def update_figure(playerid):
    one_athlete = df2[df2['meta__person__unique_id'].str.contains(playerid)]
    df_hips = one_athlete.filter(regex=re.compile(r'meta__person__unique_id|meta__session__session_guid|timestamp|hip|HIP|Hip'))

    lines = []

    for column in df_hips.columns[5:20]:
        trace = go.Scatter(
            x=df_hips['timestamp'],
            y=df_hips[column],
            mode='lines',
            name=column
        )
        lines.append(trace)

    layout = go.Layout(
        title=f"Clinician Hips",
        xaxis=dict(title='Timestamp'),
        yaxis=dict(title='Value')
    )

    # Create the figure and plot it
    fig = go.Figure(data=lines, layout=layout)
    return fig

if __name__ == '__main__':
    app.run_server(host = "0.0.0.0", debug=True, port=8050)